In [1]:
import numpy as np

In [2]:
def error(W, A):
    WtW = W.T.dot(W)
    AtA = A.T.dot(A)
    AtA1 = np.linalg.pinv(AtA)
    prof = np.trace(WtW.dot(AtA1))
    delta = np.abs(A).sum(axis=0).max()
    return delta**2 * prof

In [3]:
I = np.eye(5)
O = np.ones((5,5))
T = np.ones(5)
M = np.vstack([T,I])
W = 1 - np.kron(1 - I, 1 - I)
A = np.kron(I, I)
B = np.kron(M, M)
C = np.vstack([A, np.kron(T,T)])
D = np.random.rand(25, 25)
E = np.vstack([np.kron(I,T), np.kron(T,I)])
print error(W, A), error(W, B), error(W, C), error(W, W), error(W, D), error(W, E)
#print W.T.dot(W)

225.0 633.333333333 588.461538462 2025.0 564328.149937 134.8


In [4]:
# 1 - (1 - I) x (1 - I)

def disjunct_error(W1, W2, A, B):
    W1 = 1 - W1
    W2 = 1 - W2
    O1 = np.ones_like(W1)
    O2 = np.ones_like(W2)
    
    WtW11 = O1.T.dot(O1)
    WtW21 = O2.T.dot(O2)
    
    WtW12 = O1.T.dot(W1)
    WtW22 = O2.T.dot(W2)
    
    WtW13 = W1.T.dot(O1)
    WtW23 = W2.T.dot(O2)
    
    WtW14 = W1.T.dot(W1)
    WtW24 = W2.T.dot(W2)
    
    AtA1 = np.linalg.pinv(A.T.dot(A))
    BtB1 = np.linalg.pinv(B.T.dot(B))
    
    t1 = np.trace(WtW11.dot(AtA1)) * np.trace(WtW21.dot(BtB1))
    t2 = np.trace(WtW12.dot(AtA1)) * np.trace(WtW22.dot(BtB1))
    t3 = np.trace(WtW13.dot(AtA1)) * np.trace(WtW23.dot(BtB1))
    t4 = np.trace(WtW14.dot(AtA1)) * np.trace(WtW24.dot(BtB1))
    
    return t1-t2-t3+t4

def disjunct_expand(Ws):
    d = len(Ws)
    WtWs = []
    for i in range(len(Ws)):
        W = 1 - Ws[i]
        O = np.ones_like(W)
        WtW1 = O.T.dot(O)
        WtW2 = O.T.dot(W)
        WtW3 = W.T.dot(O)
        WtW4 = W.T.dot(W)
        WtWs.append([WtW1, WtW2, WtW3, WtW4])
    return map(list, zip(*WtWs))
        

In [103]:
I = np.eye(5)
R = np.random.rand(5,5)
R /= R.sum(axis=0)

W = 1 - np.kron(1-I, 1-I)
print error(W, np.kron(R,R)), disjunct_error(I, I, R,R)
disjunct_expand([I,I])

48656.9222218 48656.9222218


[[array([[ 5.,  5.,  5.,  5.,  5.],
         [ 5.,  5.,  5.,  5.,  5.],
         [ 5.,  5.,  5.,  5.,  5.],
         [ 5.,  5.,  5.,  5.,  5.],
         [ 5.,  5.,  5.,  5.,  5.]]), array([[ 5.,  5.,  5.,  5.,  5.],
         [ 5.,  5.,  5.,  5.,  5.],
         [ 5.,  5.,  5.,  5.,  5.],
         [ 5.,  5.,  5.,  5.,  5.],
         [ 5.,  5.,  5.,  5.,  5.]])], [array([[ 1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.]]), array([[ 1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.]])], [array([[ 1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.]]), array([[ 1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,

In [8]:
import optimize
n = 8
I = np.eye(n)
WtWs = disjunct_expand([I,I])
p = 1

init = [np.random.rand(p, n) for _ in range(4)]
ans = optimize.union_kron(WtWs, init)
print ans['As'][0][-p:]

A1, A2 = ans['As']
AtA1 = np.linalg.pinv(A1.T.dot(A1))
BtB1 = np.linalg.pinv(A2.T.dot(A2))
res = 0
res += np.trace(WtWs[0][0].dot(AtA1)) * np.trace(WtWs[0][1].dot(BtB1))
res -= np.trace(WtWs[1][0].dot(AtA1)) * np.trace(WtWs[1][1].dot(BtB1))
res -= np.trace(WtWs[2][0].dot(AtA1)) * np.trace(WtWs[2][1].dot(BtB1))
res += np.trace(WtWs[3][0].dot(AtA1)) * np.trace(WtWs[3][1].dot(BtB1))

W = 1 - np.kron(1-I, 1-I)
A = reduce(np.kron, ans['As'])

WtWs[1][0] *= -1
WtWs[2][0] *= -1

res2 = 0
res2 += np.trace(WtWs[0][0].dot(AtA1)) * np.trace(WtWs[0][1].dot(BtB1))
res2 += np.trace(WtWs[1][0].dot(AtA1)) * np.trace(WtWs[1][1].dot(BtB1))
res2 += np.trace(WtWs[2][0].dot(AtA1)) * np.trace(WtWs[2][1].dot(BtB1))
res2 += np.trace(WtWs[3][0].dot(AtA1)) * np.trace(WtWs[3][1].dot(BtB1))

print ans['error'], error(W, A), res, disjunct_error(I, I, A1, A2), res2

[[ 0.54383106  0.5438314   0.5438312   0.54383133  0.54383109  0.5438309
   0.54383134  0.54383107]]
4339.39268757 2446.25106747 2446.25106747 2446.25106747 2446.25106747
